Created 3/29/2023 for bettering the promoter/inhibitor split when training BERT

# Data Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
MODEL_PATH = "/content/drive/MyDrive/Colab Notebooks/NLP - Lab/WordEmbeddings/Models/Extracted Sentences/"

In [ ]:
import json

In [ ]:
with open(MODEL_PATH + "all_years_array.json", "r") as f:
    all_docs = json.load(f)

In [ ]:
len(all_docs)

700

In [ ]:
all_docs_1D = [j for sub in all_docs for j in sub]

In [ ]:
len(all_docs_1D)

755496

### Seeing what molecules we can use from manually labeled dataset into our training

In [ ]:
import pandas as pd
import numpy as np 

In [ ]:
bert_scores_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/NLP - Lab/GraphSage/bert_regen_score_dict_supervised_3_classes_no_molecules.csv")

In [ ]:
bert_scores_df

,Unnamed: 0,% Inhibitor,% Promoter,% Neither,Total Sentences Found
0,iopn,0.046197,0.697341,0.256462,1
1,igf-i,0.003653,0.091289,0.905058,1
2,gfra,0.244903,0.748698,0.006400,4
3,adam,0.237668,0.185988,0.576344,5
4,mirna-21,0.999660,0.000159,0.000181,1
...,...,...,...,...,...
817,sema3a,0.846462,0.036098,0.117440,240
818,igf1,0.543830,0.240735,0.215435,11
819,nogo66,0.794675,0.089106,0.116219,10
820,shh,0.462577,0.200123,0.337300,79


In [ ]:
CK_molecules = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/NLP - Lab/GraphSage/ManuallyLabeledMolecules/CK_MoleculesToLabel_JJ.xlsx")
AH_molecules = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/NLP - Lab/GraphSage/ManuallyLabeledMolecules/AH_MoleculesLabeled.xlsx")
AC_molecules = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/NLP - Lab/GraphSage/ManuallyLabeledMolecules/AC_MoleculesToLabel_JJ.xlsx")

In [ ]:
# Ensure all columns have same rows 
mask_CK = bert_scores_df.iloc[:,0] == CK_molecules.iloc[:,0]
mask_AC = bert_scores_df.iloc[:,0] == AH_molecules.iloc[:,0]
mask_AH = bert_scores_df.iloc[:,0] == AH_molecules.iloc[:,0]

print(list(mask_AH) == list(mask_CK))
print(list(mask_AC) == list(mask_CK))
print(list(mask_AH) == list(mask_AC))

True
True
True


In [ ]:
# These are the molecules that need to be relabeled since greek letters 
# didn't get converted properly 
molecules_to_relabel_BERT = list(bert_scores_df[mask_CK == False].iloc[:,0])
molecules_to_relabel_excel = list(CK_molecules[mask_CK == False].iloc[:,0])
bert_scores_df[mask_CK == False]

,Unnamed: 0,% Inhibitor,% Promoter,% Neither,Total Sentences Found
10,ifnγ,0.875249,0.123310,0.001441,5
38,tgf-β,0.478886,0.175836,0.345278,15
52,tnf-α,0.314417,0.388922,0.296661,18
57,gsk3β,0.699657,0.194657,0.105686,89
75,gsk-3β,0.942369,0.056762,0.000868,13
122,gmf-,0.112389,0.456985,0.430626,4
133,gad- d45α,0.999442,0.000225,0.000333,1
215,tgf-(1,0.001200,0.024280,0.974519,1
311,pkcα,0.051267,0.946319,0.002414,2
317,gfap –/–,0.996635,0.003158,0.000208,1


In [ ]:
# Check if any duplicate labeled molecules across sheets 
CK_indeces = CK_molecules.loc[:,"Promoter/Inhibtior"][CK_molecules.loc[:,"Promoter/Inhibtior"].isnull() == False].index
AH_indeces = CK_molecules.loc[:,"Promoter/Inhibtior"][AH_molecules.loc[:,"Promoter/Inhibtior"].isnull() == False].index
AC_indeces = CK_molecules.loc[:,"Promoter/Inhibtior"][AC_molecules.loc[:,"Promoter/Inhibtior"].isnull() == False].index

# Only 1 and 4 are common among all sheets which is fine since that must have been pre-labeled 
print(set(CK_indeces) & set(AH_indeces))
print(set(CK_indeces) & set(AC_indeces))
print(set(AC_indeces) & set(AH_indeces))

{1, 4}
{1, 4}
{1, 4}


In [ ]:
CK_molecules.loc[:,"Abbreviated Name"]

0          iopn
1         igf-i
2          gfra
3          adam
4      mirna-21
         ...   
817      sema3a
818        igf1
819      nogo66
820         shh
821         atf
Name: Abbreviated Name, Length: 822, dtype: object

In [ ]:
# Concatenate the non-null values of all excel sheets 
non_null_CK = CK_molecules.loc[:,["Abbreviated Name", "Promoter/Inhibtior"]][CK_molecules.loc[:,"Promoter/Inhibtior"].isnull() == False]
non_null_AC = AC_molecules.loc[:,["Abbreviated Name", "Promoter/Inhibtior"]][AC_molecules.loc[:,"Promoter/Inhibtior"].isnull() == False]
non_null_AH = AH_molecules.loc[:,["Abbreviated Name", "Promoter/Inhibtior"]][AH_molecules.loc[:,"Promoter/Inhibtior"].isnull() == False]

# Validate by ensuring lenghth adds up 
print(len(non_null_CK) + len(non_null_AC) + len(non_null_AH))

all_man_labeled = pd.concat([non_null_CK, non_null_AC, non_null_AH])
print(len(all_man_labeled))

141
141


In [ ]:
all_man_labeled_plus_total_sentences = all_man_labeled.merge(bert_scores_df, left_on='Abbreviated Name', right_on='Unnamed: 0')
all_man_labeled_plus_total_sentences

,Abbreviated Name,Promoter/Inhibtior,Unnamed: 0,% Inhibitor,% Promoter,% Neither,Total Sentences Found
0,igf-i,Promoter,igf-i,0.003653,0.091289,0.905058,1
1,igf-i,Promoter,igf-i,0.003653,0.091289,0.905058,1
2,igf-i,Promoter,igf-i,0.003653,0.091289,0.905058,1
3,mirna-21,Promoter,mirna-21,0.999660,0.000159,0.000181,1
4,mirna-21,Promoter,mirna-21,0.999660,0.000159,0.000181,1
...,...,...,...,...,...,...,...
134,creb,Promoter,creb,0.374481,0.510822,0.114697,29
135,bpv,Promoter?,bpv,0.830725,0.166560,0.002715,6
136,netrin,Promoter,netrin,0.039063,0.915016,0.045921,26
137,ntfs,Promoter,ntfs,0.521509,0.220417,0.258074,3


In [ ]:
all_man_labeled_plus_total_sentences.sort_values(by="Total Sentences Found", ascending=False).head(n=40)

,Abbreviated Name,Promoter/Inhibtior,Unnamed: 0,% Inhibitor,% Promoter,% Neither,Total Sentences Found
82,mag,Inhibitor,mag,0.886303,0.070692,0.043005,524
138,osm,No effect (increases excitability),osm,0.042739,0.073812,0.883450,520
80,ipsi,Niether,ipsi,0.158849,0.110719,0.730432,457
19,gfp,promoter,gfp,0.335086,0.198528,0.466386,392
62,pten,Inhibitor,pten,0.894349,0.043147,0.062504,363
10,fgf-2,promoter,fgf-2,0.130378,0.579210,0.290412,227
113,ngr1,Inhibitor,ngr1,0.821050,0.038467,0.140483,217
125,abc,No effect (label),abc,0.611699,0.109629,0.278673,191
27,chabc,promoter,chabc,0.607260,0.221366,0.171374,164
95,socs3,Inhibit (degrade),socs3,0.791075,0.081719,0.127207,154


In [ ]:
new_promoters = ['gfp', 'fgf-2', 'chabc', 'map', 'ntn', 'gal-1', 'fgf2', 'app', 'ac', 'hs']
total_new_promoter_sentences = 0
for m in new_promoters: 
  num_sentences = all_man_labeled_plus_total_sentences[all_man_labeled_plus_total_sentences.iloc[:,0] == m].loc[:,'Total Sentences Found'].values[0]
  print(num_sentences)
  total_new_promoter_sentences += num_sentences

print('total new promoter sentences:', total_new_promoter_sentences)

392
227
164
150
124
118
105
95
92
89
total new promoter sentences: 1556


---

In [ ]:
# Add new promoters to list (from below section where we found it ) 

orig_molecules = ["netrin", "ephrin", "laminin", "tenascin", "cspg", "zymosan", "camp", "pten", "cntf", "lif", "oncomodulin", "stat3", "socs3", "rhoa", "rock", "y27632", "nogo", "klf", "ngr", "lar", "tlr2", "bdnf", "igf1", "opn", "mag", "omgp", "kspg", "taxol", ]
orig_molecules += new_promoters 
molecule_labels = [1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1] 
molecule_labels += [1] * len(new_promoters)

to_remove = orig_molecules.index("lar")
orig_molecules.pop(to_remove) 
molecule_labels.pop(to_remove)

label_dict = {k:v for (k,v) in zip(orig_molecules, molecule_labels)}
label_dict

{'netrin': 1,
 'ephrin': 0,
 'laminin': 1,
 'tenascin': 0,
 'cspg': 0,
 'zymosan': 1,
 'camp': 1,
 'pten': 0,
 'cntf': 1,
 'lif': 1,
 'oncomodulin': 1,
 'stat3': 1,
 'socs3': 0,
 'rhoa': 0,
 'rock': 0,
 'y27632': 1,
 'nogo': 0,
 'klf': 1,
 'ngr': 0,
 'tlr2': 1,
 'bdnf': 0,
 'igf1': 1,
 'opn': 1,
 'mag': 0,
 'omgp': 0,
 'kspg': 0,
 'taxol': 1,
 'gfp': 1,
 'fgf-2': 1,
 'chabc': 1,
 'map': 1,
 'ntn': 1,
 'gal-1': 1,
 'fgf2': 1,
 'app': 1,
 'ac': 1,
 'hs': 1}

In [ ]:
from collections import Counter 

Counter(molecule_labels)

Counter({1: 24, 0: 13})

### Getting all molecules from abbreviation detector to later label

In [ ]:
import pandas as pd
new_molecules_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/NLP - Lab/GraphSage/Named_Entity_Recognition/Abrv_Curated.csv", header=None)
new_molecules_df

,0,1,2,3,4,5
0,pi,phosphatidylinositol,NaN,NaN,NaN,NaN
1,pe,phosphatidylethanolamine,NaN,NaN,NaN,NaN
2,sm,sphingomyelin,NaN,NaN,NaN,NaN
3,hrp,horseradish peroxidase,NaN,NaN,NaN,NaN
4,ritc,rhodamine isothiocyanate,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
1075,ma,methacrylate,NaN,NaN,NaN,NaN
1076,moetacl,hema-2[2-(methacryloyloxy)ethyl]trimethylammon...,NaN,NaN,NaN,NaN
1077,th-,tyrosine hydroxylase-,NaN,NaN,NaN,NaN
1078,phb,poly-beta-hydroxybutyrate,NaN,NaN,NaN,NaN


In [ ]:
new_molecules = list(set(new_molecules_df.loc[:,0]))

In [ ]:
new_molecules_df[new_molecules_df[0] == 'if']

,0,1,2,3,4,5
943,if,intermediate filament s,NaN,NaN,NaN,NaN


In [ ]:
new_molecules.remove("if")
new_molecules.remove("of")
new_molecules.remove("am")
new_molecules.remove("fat")
new_molecules.remove("th")
new_molecules.remove("th-")
new_molecules.remove("m.")
new_molecules.remove("set")
new_molecules.remove("protein")

In [ ]:
len(new_molecules)

813

# Generating Supervised Sentences only from Orig Molecules


In [ ]:
# make sure to initialize this AFTER removing molecules with no embeddings
orig_molecules_double_spaced = [" " + m + " " for m in orig_molecules]
print(orig_molecules_double_spaced)

[' netrin ', ' ephrin ', ' laminin ', ' tenascin ', ' cspg ', ' zymosan ', ' camp ', ' pten ', ' cntf ', ' lif ', ' oncomodulin ', ' stat3 ', ' socs3 ', ' rhoa ', ' rock ', ' y27632 ', ' nogo ', ' klf ', ' ngr ', ' tlr2 ', ' bdnf ', ' igf1 ', ' opn ', ' mag ', ' omgp ', ' kspg ', ' taxol ', ' gfp ', ' fgf-2 ', ' chabc ', ' map ', ' ntn ', ' gal-1 ', ' fgf2 ', ' app ', ' ac ', ' hs ']


In [ ]:
# make sure to initialize this AFTER removing molecules with no embeddings
new_molecules_double_spaced = [" " + m + " " for m in new_molecules]
print(new_molecules_double_spaced)

[' psa ', ' tlr ', ' hme ', ' phema ', ' npn-1 ', ' chat ', ' tgf-\uf0621 ', ' civ ', ' c-myc ', ' saps ', ' irf5 ', ' myti ', ' lama2 ', ' sreb ', ' ascl1 ', ' ccc ', ' aqp ', ' cpt-camp ', ' pkg ', ' plla ', ' c3 ', ' sod ', ' rho gdi ', ' c/ebp-β ', ' cs-gag ', ' sema 3a ', ' acs ', ' nf-kb ', ' psa-ncam ', ' pak1 ', ' bmp-7 ', ' erk1/2 ', ' crem ', ' mtor ', ' fgfs ', ' ampa ', ' tbst ', ' pvdf ', ' irs-1 ', ' igf-1 ', ' cre+wt/wt ', ' am-6 ', ' tgfa ', ' bcan ', ' vla-4 ', ' blys ', ' ulip ', ' syncams ', ' pan/ pvc ', ' nrdg1 ', ' pdgf-bb ', ' magr ', ' wga-hrp ', ' o4 ', ' page ', ' crmp2 ', ' tnfbp ', ' hmg-1 ', ' cre ', ' dmso ', ' ifn-γ ', ' co ', ' tgf-(1 ', ' il-1 ', ' tmrd ', ' tlr-4 ', ' sybi ', ' gdf11 ', ' ttn ', ' crmp-2 ', ' a\uf062 ', ' igfs ', ' p-tubulin ', ' hb-egf ', ' npc ', ' krx-20 ', ' atl ', ' gsk-3β ', ' stat6 ', ' mfps ', ' plg ', ' lane 14 ', ' pfn ', ' igf ', ' gsk3 ', ' ppars ', ' tsc1/2 ', ' eph ', ' pdgfr ', ' tnfa ', ' pi3k ', ' edta ', ' ppc ', ' pd

In [ ]:
stop_code

NameError: ignored

In [ ]:
# from collections import defaultdict
# # 0 index will be num of inihibitors, 1st index will be number of promoters 
# trainable_sentences = defaultdict(int)
# for counter, sentence in enumerate(all_docs_1D):
#   promoter = False 
#   inhibitor = False
#   for molecule in orig_molecules_double_spaced:
#     if molecule in sentence:
#       molecule_label = label_dict[molecule.strip()]
#       if molecule_label == 0:
#         inhibitor = True 
#       else:
#         promoter = True 

#   to_label = 0 
#   if promoter and inhibitor:
#     continue 
#   if not promoter and not inhibitor:
#     found_new_molecule = False
#     for new_molecule in new_molecules_double_spaced:
#       if new_molecule in sentence:
#         found_new_molecule = True 
#         break 
#     if found_new_molecule:
#       # print(f"{counter}/{len(all_docs_1D)}")
#       # print("--> Skipping sentence found new molecule")
#       continue # don't label if there's one of the new molecules in there
#     else:
#       to_label = 2 # if no new molecule then can label as neutral 
#   if promoter and not inhibitor:
#     to_label = 1
#   elif not promoter and inhibitor:
#     to_label = 0 

#   # Remove molecules from sentence to better train bert on other words 
#   # of sentence 
#   sentencewords = sentence.split()
#   resultwords  = [word for word in sentencewords if word.lower() not in orig_molecules]
#   result = ' '.join(resultwords)
#   trainable_sentences[result.strip()] = to_label

#   if to_label == 0 or to_label == 1:
#     print(f"{counter}/{len(all_docs_1D)}")
#     print(f"--> P:{promoter} and I:{inhibitor} found in {result.strip()}")
#     print(f"--> Labeling sentence as {to_label}")

# trainable_sentences

In [ ]:
# import pickle
# with open("/content/drive/MyDrive/Linguistic Causation/Roberta Model/Saved Models/balanced_trainable_sentences_no_molecules.pkl", 'wb') as handle:
#     pickle.dump(trainable_sentences, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import pickle
with open("/content/drive/MyDrive/Linguistic Causation/Roberta Model/Saved Models/balanced_trainable_sentences_no_molecules.pkl", 'rb') as handle:
    trainable_sentences = pickle.load(handle)

trainable_sentences

In [ ]:
# Get only promoters and inhibitors 
trainable_sentences_prom_inhib = {key:value for key,value in trainable_sentences.items() if (value == 1 or value == 0)}
trainable_sentences_prom_inhib

# Training BioBert

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 95.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 71.8 MB/s eta 0:00:00


In [ ]:
## Model.py ## 

import transformers, torch
import torch.nn as nn
import torch.nn.functional as F

tokenizer = transformers.RobertaTokenizerFast.from_pretrained("roberta-base")

def _tokenize(st):
    tokenized_st = tokenizer(
        st,
        max_length=64,
        padding="max_length",
        truncation=True,
    )
    return tokenized_st


DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device", DEVICE)


class TrainingModel(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.encoder = transformers.RobertaModel.from_pretrained("roberta-base")
        self.linear = nn.Linear(self.encoder.config.hidden_size, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, batch):
        encoder_out = self.encoder(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
        )
        logits = self.linear(encoder_out["pooler_output"])
        output = self.softmax(logits)
        result = {"output": output}
        if "labels" in batch:
            result["loss"] = F.cross_entropy(output, batch["labels"])
        return result


class InferenceModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = transformers.RobertaModel.from_pretrained("roberta-base")
        self.linear = nn.Linear(self.encoder.config.hidden_size, 2)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, tokenized_comment):
        encoder_out = self.encoder(
            input_ids = torch.as_tensor(tokenized_comment["input_ids"]).unsqueeze(0).to(DEVICE),
            attention_mask = torch.as_tensor(tokenized_comment["attention_mask"]).unsqueeze(0).to(DEVICE),
        )
        logits = self.linear(encoder_out["pooler_output"])
        return self.softmax(logits)


class Predict:
    def __init__(self, model_name):
        self.model = InferenceModel()
        self.model.load_state_dict(torch.load(model_name, map_location=DEVICE))
        self.model = self.model.to(DEVICE)
        self.model.eval()

    def process_eval(self, s):
        # s = strip_tags(s)
        tokens = _tokenize(s)
        return tokens


    def predict(self, comment):
        tokenized_comment = self.process_eval(comment)
        with torch.no_grad():
            logits = self.model(tokenized_comment)
        confidence = float(logits[0][1])
        return logits

Using device cuda


In [ ]:
from torch.utils.data import DataLoader, Dataset

class Data(Dataset):
# class Data(list):
    def __init__(self, X, y):
        assert len(X) == len(y)
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return_item = _tokenize(self.X[idx])
        return_item["label"] = self.y[idx]

        return return_item

In [ ]:
# import numpy as np 
# import collections
# values_arr = list(trainable_sentences_prom_inhib.values())
# counter = collections.Counter(values_arr)
# counter # way more balanced promtoer and inhibitor split! 

In [ ]:
# from sklearn import tree
# from sklearn.model_selection import train_test_split

# features_arr = list(trainable_sentences_prom_inhib.keys())
# targets_arr = list(trainable_sentences_prom_inhib.values())
# X_train, X_test, y_train, y_test = train_test_split(features_arr, targets_arr, test_size=0.20)

# y_train = [int(x) for x in y_train]
# y_test = [int(x) for x in y_test]

# train_dataset = Data(X_train, y_train)
# val_dataset = Data(X_test, y_test)

In [ ]:
# ## train.py ## 

# import numpy as np
# import torch
# from transformers import AdamW
# from tqdm import tqdm
# # from Model import TrainingModel
# import transformers
# from sklearn.metrics import roc_auc_score
# # from plot_results import plot


# num_epochs = 10
# DOWNSAMPLE = True
# NARROW = None #"UCOMrUmOTPD_AnSivjxptxpA" #Louis
# IGNORE_WHALE = False
# model_tag = 'thoughtful_channels'


# DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print("Using device", DEVICE)

# def to_device(batch):
#     return {k: v.to(DEVICE) for k, v in batch.items()}


# # train_dataloader, val_dataloader = get_data_main(NARROW, DOWNSAMPLE, IGNORE_WHALE)
# train_dataloader = DataLoader(
#         train_dataset,
#         batch_size=32,
#         shuffle=True,
#         collate_fn=transformers.data.DataCollatorWithPadding(tokenizer),
#     )

# val_dataloader = DataLoader(
#         val_dataset,
#         batch_size=32,
#         shuffle=False,
#         collate_fn=transformers.data.DataCollatorWithPadding(tokenizer),
# )

# model = TrainingModel(num_classes=2).to(DEVICE)
# optimizer = AdamW(model.parameters(), lr=1e-5, weight_decay=0.01)

# scheduler = transformers.get_linear_schedule_with_warmup(
#     optimizer,
#     num_warmup_steps=100,  
#     num_training_steps=len(train_dataloader) * num_epochs,
# )


# all_auc, all_val_loss, all_train_loss = [], [], []
# for epoch in range(num_epochs):
#     train_loss_ls = []
#     model.train()
#     for batch in tqdm(train_dataloader, desc=f"train e_{epoch}"):
#         optimizer.zero_grad()
#         out = model(to_device(batch))
#         out["loss"].backward()

#         optimizer.step()
#         scheduler.step()
#         train_loss_ls.append(out["loss"].item())

#     optimizer.zero_grad()
#     train_loss = np.mean(train_loss_ls)

#     # validation
#     model.eval()
#     val_loss_ls, y_hat, y_true = [], [], []
#     for batch in tqdm(val_dataloader, desc="val"):
#         batch = to_device(batch)
#         with torch.inference_mode():
#             out = model(batch)
        
#         output = out['output'].cpu().numpy()
#         y_hat.append(output[:, 1]) ## confidence for a positive sample
        
#         # preds = out["output"].argmax(-1).detach()
#         # y_pred.append(preds.cpu().float().numpy())
#         y_true.append(batch["labels"].cpu().float().numpy())
#         val_loss_ls.append(out["loss"].item())

#     y_true, y_hat = np.concatenate(y_true), np.concatenate(y_hat)
#     auc = roc_auc_score(y_true, y_hat) if len(np.unique(y_true)) > 1 else -1
#     val_loss = np.mean(val_loss_ls)

#     all_auc.append(auc)
#     all_val_loss.append(val_loss)
#     all_train_loss.append(train_loss)

#     print(f"Epoch {epoch}: AUC={auc:.2f}, Train Loss={train_loss:.2f}, Val Loss={val_loss:.2f}\n")

In [ ]:
all_auc

In [ ]:
# all_f1_macro

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# plt.plot(list(range(num_epochs)), all_f1_macro) 

In [ ]:
torch.save(model.state_dict(), "/content/drive/MyDrive/Linguistic Causation/Roberta Model/Saved Models/Balanced_Supervised_2_classes_no_molecules_10_epochs.pth")

In [ ]:
import matplotlib.pyplot as plt
plt.plot(list(range(num_epochs)), all_auc) 

In [ ]:
plt.plot(list(range(num_epochs)), all_val_loss) 

In [ ]:
plt.plot(list(range(num_epochs)), all_train_loss)

In [ ]:
p = Predict("/content/drive/MyDrive/Linguistic Causation/Roberta Model/Saved Models/Balanced_Supervised_2_classes_no_molecules_10_epochs.pth") 
# model.load_state_dict(torch.load(bert_model_path))
# model.eval()

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
p.predict("with camp we can accelerate axon growth") 

(tensor([[0.9787, 0.0213]], device='cuda:0'), 0.02126649208366871)

In [ ]:
p.predict("we show that this is a strong promoter")

(tensor([[0.2283, 0.7717]], device='cuda:0'), 0.7717058062553406)

# Creating Bert Regen Scores 2 Classes for All Molecules

In [ ]:
p.predict("we show that xre is a strong inhibitor") 

In [ ]:
p.predict("pbs is a saline") 

In [ ]:
p.predict("nmo does not grow axons") 

In [ ]:
p.predict("nmo increases length of axons") 

In [ ]:
# Predict on all molecules 
molecules = orig_molecules + new_molecules
molecules = list((set(molecules)))
molecules_double_spaced = [" " + m + " " for m in molecules]
molecules_double_spaced
molecules_double_spaced.remove(" no ")


In [ ]:
import numpy as np

In [ ]:
p.predict("we show that this is a strong promoter").detach().cpu().numpy() + np.array([1.0, 1.0])

array([[1.22829422, 1.77170581]])

In [ ]:
bert_regen_score_dict_supervised = {} 

for counter, keyword in enumerate(molecules_double_spaced):
  print(f"{counter} / {len(molecules)} Current keyword: " + keyword + "...")
  keyword_regen_score = np.array([0.,0.])
  total_sentences_found = 0
  for sentence in all_docs_1D:
    if keyword in sentence:
      keyword_regen_score = p.predict(sentence).detach().cpu().numpy() + keyword_regen_score
      total_sentences_found += 1
  
  if total_sentences_found == 0:
    # redo but with no padding spaces around molecule
    for sentence in all_docs_1D:
      if keyword.strip() in sentence:
        keyword_regen_score = p.predict(sentence).detach().cpu().numpy() + keyword_regen_score
        total_sentences_found += 1

  column = [keyword_regen_score / total_sentences_found, total_sentences_found] 
  bert_regen_score_dict_supervised[keyword.strip()] = column
  print("    Calculated score of {}".format(keyword_regen_score / total_sentences_found)) 
  print("    Total Sentences found: {}".format(total_sentences_found)) 

0 / 823 Current keyword:  psa ...
    Calculated score of [[0.84001534 0.15998466]]
    Total Sentences found: 14
1 / 823 Current keyword:  tlr ...
    Calculated score of [[0.74632492 0.2536751 ]]
    Total Sentences found: 6
2 / 823 Current keyword:  hme ...
    Calculated score of [[0.34367515 0.65632485]]
    Total Sentences found: 1091
3 / 823 Current keyword:  phema ...
    Calculated score of [[0.22781424 0.77218579]]
    Total Sentences found: 9
4 / 823 Current keyword:  npn-1 ...
    Calculated score of [[0.84104641 0.1589536 ]]
    Total Sentences found: 11
5 / 823 Current keyword:  chat ...
    Calculated score of [[0.28361779 0.71638222]]
    Total Sentences found: 13
6 / 823 Current keyword:  tgf-1 ...
    Calculated score of [[4.70225685e-04 9.99529819e-01]]
    Total Sentences found: 3
7 / 823 Current keyword:  civ ...
    Calculated score of [[0.09482885 0.90517115]]
    Total Sentences found: 11
8 / 823 Current keyword:  c-myc ...
    Calculated score of [[0.70636555 

In [ ]:
import pickle 
bert_regen_scores_path = "/content/drive/MyDrive/Colab Notebooks/NLP - Lab/GraphSage/bert_regen_score_dict_supervised_2_classes_balanced_5_5_23.csv"
with open(bert_regen_scores_path, 'wb') as f:
    pickle.dump(bert_regen_score_dict_supervised, f)

# Evaluate Performance

Based on the below colab file but modified for two classes rather than 3:  
https://colab.research.google.com/drive/1Pc_Pq5qxuPqOAvKjJggGsIjISD7hM_bK

In [ ]:
import pickle 
bert_regen_scores_path = "/content/drive/MyDrive/Colab Notebooks/NLP - Lab/GraphSage/bert_regen_score_dict_supervised_2_classes_balanced_5_5_23.csv"
bert_regen_scores_loaded = {} 
with open(bert_regen_scores_path, 'rb') as f:
  bert_regen_scores_loaded = pickle.load(f)

In [ ]:
# Convert to format used to evaluate from above file 

In [ ]:
df = pd.DataFrame(bert_regen_scores_loaded).T
df.columns = ['Avg Bert Regen Score', 'Total Sentences Found']
df

,Avg Bert Regen Score,Total Sentences Found
psa,"[[0.8400153383180233, 0.15998466100635206]]",14
tlr,"[[0.7463249154388905, 0.2536750996951014]]",6
hme,"[[0.34367514612639694, 0.6563248545420042]]",1091
phema,"[[0.22781423563396352, 0.772185785902871]]",9
npn-1,"[[0.8410464089247398, 0.15895359646211463]]",11
...,...,...
ankrd1,"[[0.999631404876709, 0.0003685659321490675]]",1
mmp,"[[0.7815456638604196, 0.2184543252919361]]",9
nadph,"[[0.2729603885400138, 0.7270395946834469]]",8
dbi,"[[0.5385938286781311, 0.4614061140891863]]",2


In [ ]:
df['% Inhibitor'] = df.apply(lambda row: row[0][0][0], axis=1)
df['% Promoter'] = df.apply(lambda row: row[0][0][1], axis=1)

In [ ]:
df = df[['% Inhibitor', '% Promoter', 'Total Sentences Found']]

In [ ]:
df.to_csv("/content/drive/MyDrive/Colab Notebooks/NLP - Lab/GraphSage/bert_regen_score_dict_supervised_2_classes_balanced_5_5_23.csv")

In [ ]:
bert_scores_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/NLP - Lab/GraphSage/bert_regen_score_dict_supervised_2_classes_balanced_5_5_23.csv")

In [ ]:
bert_scores_df

,Unnamed: 0,% Inhibitor,% Promoter,Total Sentences Found
0,psa,0.840015,0.159985,14
1,tlr,0.746325,0.253675,6
2,hme,0.343675,0.656325,1091
3,phema,0.227814,0.772186,9
4,npn-1,0.841046,0.158954,11
...,...,...,...,...
817,ankrd1,0.999631,0.000369,1
818,mmp,0.781546,0.218454,9
819,nadph,0.272960,0.727040,8
820,dbi,0.538594,0.461406,2


In [ ]:
CK_molecules = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/NLP - Lab/GraphSage/ManuallyLabeledMolecules/CK_MoleculesToLabel_JJ.xlsx")
AH_molecules = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/NLP - Lab/GraphSage/ManuallyLabeledMolecules/AH_MoleculesLabeled.xlsx")
AC_molecules = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/NLP - Lab/GraphSage/ManuallyLabeledMolecules/AC_MoleculesToLabel_JJ.xlsx")

In [ ]:
CK_molecules

,Abbreviated Name,Full Name,Promoter/Inhibtior,Unnamed: 3,Unnamed: 4
0,iopn,intracellular osteopontin,NaN,NaN,NaN
1,igf-i,"insulin-like growth factor i, insulin-like gro...",Promoter,PMID: 31122676,NaN
2,gfra,gdnf family receptor a),NaN,NaN,NaN
3,adam,a disintegrin and metalloprotease,NaN,NaN,NaN
4,mirna-21,microrna-21,Promoter,PMID: 29709341,PMID: 32599451
...,...,...,...,...,...
817,sema3a,semaphorin-3a,NaN,NaN,NaN
818,igf1,NaN,NaN,NaN,NaN
819,nogo66,NaN,NaN,NaN,NaN
820,shh,"sonic hedgehog, sonic hedgehog",NaN,NaN,NaN


In [ ]:
# Ensure all columns have same rows 
mask_CK = bert_scores_df.iloc[:,0] == CK_molecules.iloc[:,0]
mask_AC = bert_scores_df.iloc[:,0] == AH_molecules.iloc[:,0]
mask_AH = bert_scores_df.iloc[:,0] == AH_molecules.iloc[:,0]

print(list(mask_AH) == list(mask_CK))
print(list(mask_AC) == list(mask_CK))
print(list(mask_AH) == list(mask_AC))

True
True
True


In [ ]:
# ! NOTE: WE can't do the below because the BERT scores df and the manually labelled df don't have the molecules 
# in the same order... 

# These are the molecules that need to be relabeled since greek letters 
# didn't get converted properly 
# molecules_to_relabel_BERT = list(bert_scores_df[mask_CK == False].iloc[:,0])
# molecules_to_relabel_excel = list(CK_molecules[mask_CK == False].iloc[:,0])
# bert_scores_df[mask_CK == False]

In [ ]:
# Check if any duplicate labeled molecules across sheets 
CK_indeces = CK_molecules.loc[:,"Promoter/Inhibtior"][CK_molecules.loc[:,"Promoter/Inhibtior"].isnull() == False].index
AH_indeces = CK_molecules.loc[:,"Promoter/Inhibtior"][AH_molecules.loc[:,"Promoter/Inhibtior"].isnull() == False].index
AC_indeces = CK_molecules.loc[:,"Promoter/Inhibtior"][AC_molecules.loc[:,"Promoter/Inhibtior"].isnull() == False].index

# Only 1 and 4 are common among all sheets which is fine since that must have been pre-labeled 
print(set(CK_indeces) & set(AH_indeces))
print(set(CK_indeces) & set(AC_indeces))
print(set(AC_indeces) & set(AH_indeces))

{1, 4}
{1, 4}
{1, 4}


In [ ]:
CK_molecules.loc[:,"Abbreviated Name"]

0          iopn
1         igf-i
2          gfra
3          adam
4      mirna-21
         ...   
817      sema3a
818        igf1
819      nogo66
820         shh
821         atf
Name: Abbreviated Name, Length: 822, dtype: object

In [ ]:
# Concatenate the non-null values of all excel sheets 
non_null_CK = CK_molecules.loc[:,["Abbreviated Name", "Promoter/Inhibtior"]][CK_molecules.loc[:,"Promoter/Inhibtior"].isnull() == False]
non_null_AC = AC_molecules.loc[:,["Abbreviated Name", "Promoter/Inhibtior"]][AC_molecules.loc[:,"Promoter/Inhibtior"].isnull() == False]
non_null_AH = AH_molecules.loc[:,["Abbreviated Name", "Promoter/Inhibtior"]][AH_molecules.loc[:,"Promoter/Inhibtior"].isnull() == False]

# Validate by ensuring lenghth adds up 
print(len(non_null_CK) + len(non_null_AC) + len(non_null_AH))

all_man_labeled = pd.concat([non_null_CK, non_null_AC, non_null_AH])
print(len(all_man_labeled))

141
141


In [ ]:
conversion_dict = {'i': 0, 'p': 1, 'n': 2}

mol_row = all_man_labeled[all_man_labeled.iloc[:,0] == "gapdh"]
conversion_dict[mol_row.loc[:,"Promoter/Inhibtior"].values[0][0].lower()]

1

In [ ]:
all_man_mol_row = all_man_labeled[all_man_labeled.iloc[:,0] == "gapdh"]
first_letter = all_man_mol_row.loc[:,"Promoter/Inhibtior"].values[0][0].lower()
conversion_dict[first_letter]

1

In [ ]:
bert_scores_df[bert_scores_df.iloc[:,0] == "fgf-2"]

,Unnamed: 0,% Inhibitor,% Promoter,Total Sentences Found
625,fgf-2,0.000271,0.999729,227


In [ ]:
conversion_dict = {'i': 0, 'p': 1, 'n': 2}
# Ensure labeling is standardized across sheets 
list(all_man_labeled.loc[:,"Promoter/Inhibtior"])

# Variation in labeling so just use first letter for Promoter/Inhibitor/Neither
# accepted_values = ["promoter", "inhibitor", "neither", "unknown", "error"]

# track breakdown of wrong results 
labeled_promoter_as_inhibitor = 0 
labeled_inhibitor_as_promoter = 0
labeled_promoter_as_neither = 0
labeled_inhibitor_as_neither = 0 
labeled_neither_as_promoter = 0 
labeled_neither_as_inhibitor = 0 

# track overall acurracy 
correct = 0
total = 0
for molecule in list(all_man_labeled.loc[:,"Abbreviated Name"]):
    # skip greek letter ones to be relabeled 
    # if molecule in molecules_to_relabel_excel: 
    #   print("{} skipped".format(molecule))
    #   continue 
 
    BERT_mol_row = bert_scores_df[bert_scores_df.iloc[:,0] == molecule]
    if BERT_mol_row.values.flatten().tolist()[1:3] == []: # mis-labeled greek one 
      continue 
    BERT_mol_label = np.argmax(BERT_mol_row.values.flatten().tolist()[1:3]) # 0: Inhib, 1: Prom, 2: Neith  
    
    all_man_mol_row = all_man_labeled[all_man_labeled.iloc[:,0] == molecule]
    first_letter = all_man_mol_row.loc[:,"Promoter/Inhibtior"].values[0][0].lower()
    # Skip if not standardized response
    if first_letter not in conversion_dict:
      continue 

    all_man_mol_label = conversion_dict[first_letter]

    if BERT_mol_label == all_man_mol_label:
      correct += 1
    elif BERT_mol_label == 0 and all_man_mol_label == 1:
      labeled_promoter_as_inhibitor += 1
    elif BERT_mol_label == 1 and all_man_mol_label == 0:
      labeled_inhibitor_as_promoter += 1
    elif BERT_mol_label == 2 and all_man_mol_label == 1:
      labeled_promoter_as_neither += 1
    elif BERT_mol_label == 2 and all_man_mol_label == 0:
      labeled_inhibitor_as_neither += 1
    elif BERT_mol_label == 1 and all_man_mol_label == 2:
      labeled_neither_as_promoter += 1
    elif BERT_mol_label == 0 and all_man_mol_label == 2:
      labeled_neither_as_inhibitor += 1

    total += 1
    print("{}: MAN_LABELED: {}, BERT: {}".format(molecule, all_man_mol_label, BERT_mol_label))

igf-i: MAN_LABELED: 1, BERT: 1
mirna-21: MAN_LABELED: 1, BERT: 0
gapdh: MAN_LABELED: 1, BERT: 1
pge2: MAN_LABELED: 1, BERT: 1
plp-1: MAN_LABELED: 1, BERT: 0
sybii: MAN_LABELED: 1, BERT: 1
fgf-2: MAN_LABELED: 1, BERT: 1
fgf2: MAN_LABELED: 1, BERT: 1
o4: MAN_LABELED: 0, BERT: 1
pfn: MAN_LABELED: 1, BERT: 1
ctb: MAN_LABELED: 0, BERT: 1
p(tmc: MAN_LABELED: 1, BERT: 1
pgipz: MAN_LABELED: 0, BERT: 0
stat3: MAN_LABELED: 1, BERT: 1
af-1: MAN_LABELED: 1, BERT: 0
gfp: MAN_LABELED: 1, BERT: 1
tnfa: MAN_LABELED: 0, BERT: 1
cs-e: MAN_LABELED: 1, BERT: 0
igfs: MAN_LABELED: 1, BERT: 1
plap: MAN_LABELED: 1, BERT: 1
sod-1: MAN_LABELED: 1, BERT: 0
pip2: MAN_LABELED: 0, BERT: 0
emt: MAN_LABELED: 1, BERT: 1
chabc: MAN_LABELED: 1, BERT: 1
doc: MAN_LABELED: 1, BERT: 1
sybi: MAN_LABELED: 0, BERT: 1
ks: MAN_LABELED: 1, BERT: 1
ntf: MAN_LABELED: 1, BERT: 0
folr1: MAN_LABELED: 0, BERT: 1
ntn: MAN_LABELED: 1, BERT: 0
plgf: MAN_LABELED: 1, BERT: 1
soc3: MAN_LABELED: 1, BERT: 0
sema: MAN_LABELED: 1, BERT: 0
magr: 

In [ ]:
print("Correct/Total: {}/{} ({:.2f})".format(correct, total, correct/total)) 
incorrects = total - correct 

print("Labeled promoter as inhibitor: {:.2f}".format(labeled_promoter_as_inhibitor/incorrects)) 
print("Labeled inhibitor as promoter: {:.2f}".format(labeled_inhibitor_as_promoter/incorrects)) 
print("Labeled promoter as neither: {:.2f}".format(labeled_promoter_as_neither/incorrects)) 
print("Labeled inhibitor as neither: {:.2f}".format(labeled_inhibitor_as_neither/incorrects)) 
print("Labeled neither as promoter: {:.2f}".format(labeled_neither_as_promoter/incorrects)) 
print("Labeled neither as inhibitor: {:.2f}".format(labeled_neither_as_inhibitor/incorrects)) 

# Validation 
incorrects == sum([labeled_promoter_as_inhibitor, labeled_inhibitor_as_promoter, labeled_promoter_as_neither, labeled_inhibitor_as_neither, labeled_neither_as_promoter, labeled_neither_as_inhibitor])

Correct/Total: 69/134 (0.51)
Labeled promoter as inhibitor: 0.42
Labeled inhibitor as promoter: 0.22
Labeled promoter as neither: 0.00
Labeled inhibitor as neither: 0.00
Labeled neither as promoter: 0.28
Labeled neither as inhibitor: 0.09


True